In [ ]:
import os
import json
import requests

In [41]:
s = '''
Конечно! Вот простая программа "Hello, World!" на языке Python:

```python
print("Hello, World!")
```

Когда вы запустите этот код, он выведет на экран строку "Hello, World!". Если у вас есть дополнительные вопросы или вам нужно что-то еще, дайте знать!
'''
sample_code = json.dumps({"question": "Напиши программу выводящую hell world на python.", "answer" : s}, ensure_ascii=False)
sample_code

'{"question": "Напиши программу выводящую hell world на python.", "answer": "\\nКонечно! Вот простая программа \\"Hello, World!\\" на языке Python:\\n\\n```python\\nprint(\\"Hello, World!\\")\\n```\\n\\nКогда вы запустите этот код, он выведет на экран строку \\"Hello, World!\\". Если у вас есть дополнительные вопросы или вам нужно что-то еще, дайте знать!\\n"}'

In [62]:
prompt = '''
Прочитай следующую документацию и сгенерируй максимальное количество пар вопрос-ответ на основе информации из текста. Представь результат в формате JSON массива с обьектами с полями `question` и `answer`.
Не добавляй markdown выделение. Вывод должен быть машиночитаемый json. Ответы должны содержать примеры кода и выделятся с помощью markdown. Экранируй кавычки.

**Документация:**

Python — это высокоуровневый язык программирования общего назначения. Он был создан Гвидо ван Россумом и впервые выпущен в 1991 году. Python поддерживает несколько парадигм программирования, включая объектно-ориентированное, императивное и функциональное программирование. Он известен своей простотой и читаемостью кода.
Простая программа на python

print("Hello world.") 

**Формат вывода:**
[
{"question": "Что такое Python?", "answer": "Python — это высокоуровневый язык программирования общего назначения."}
{"question": "Кто создал Python?", "answer": "Гвидо ван Россум"}
{"question": "Когда был впервые выпущен Python?", "answer": "В 1991 году"}
{"question": "Какие парадигмы программирования поддерживает Python?", "answer": "Объектно-ориентированное, императивное и функциональное программирование"}
{"question": "Чем известен Python?", "answer": "Простотой и читаемостью кода"}
]
''' + sample_code



In [63]:

def split_text_into_chunks(text, split_lines = 2, min_lines = 3):
    # Разделяем текст на строки
    lines = text.split('\n')

    # Инициализируем список для хранения чанков
    chunks = []

    # Инициализируем временный список для текущего чанка
    current_chunk = []

    # Флаг для отслеживания пустых строк
    empty_line_count = 0

    for line in lines:
        if line.strip() == '':
            empty_line_count += 1
        else:
            empty_line_count = 0

        if empty_line_count == split_lines:
            # Если нашли две пустые строки подряд, добавляем текущий чанк в список чанков
            if current_chunk:
                if len(current_chunk) >= min_lines:
                    chunks.append('\n'.join(current_chunk))
                current_chunk = []
            empty_line_count = 0
        else:
            # Добавляем строку в текущий чанк
            current_chunk.append(line)

    # Добавляем последний чанк, если он не пустой
    if current_chunk:
        if len(current_chunk) >= min_lines:
            chunks.append('\n'.join(current_chunk))

    return chunks


In [64]:

#def split_text_into_chunks(text, chunk_size, overlap):
#    chunks = []
#    start = 0
#    while start < len(text):
#        end = start + chunk_size
#        chunks.append(text[start:end])
#        start = end - overlap
#    return chunks
# Функция для отправки запроса к API OpenAI
def generate_qa_pairs(text, api_key, endpoint, model_name):
    url = endpoint  # URL для API OpenAI
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": model_name,  # Используем указанную модель
        "messages": [
            {"role": "system", "content": prompt},
            {"role": "user", "content": text}
        ],
        "response_format": {"type": "json_object"}
    }
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Error {response.status_code}: {response.text}")

# Функция для обработки текстовых файлов в директории
def process_text_files(directory, api_key, endpoint, model_name, output_dir):
        for filename in os.listdir(directory):
            if filename.endswith(".odc"):
                filepath = os.path.join(directory, filename)
                with open(filepath, 'r', encoding='utf-8') as text_file:
                    text = text_file.read()
                    out = f'{output_dir}/{filename}.jsonl'
                    with open(out, 'w', encoding='utf-8') as file:
                    
                        chunks = split_text_into_chunks(text)
                        for i, chunk in enumerate(chunks):
                            try:
                                response = generate_qa_pairs(chunk, api_key, endpoint, model_name)
                                qa_pairs = response['choices'][0]['message']['content']
                                file.write(qa_pairs)
                            except Exception as e:
                                print(f"Error processing file {filepath} chunk {i}: {e}")
                print(f"Wrote {out}")
                




In [68]:

directory = "/app/datasets/oberon/docs/bb_ru/Docu/ru"  # Замените на путь к вашей директории с текстовыми файлами
api_key = "Ded77YQ7CTIhXKasMKSPNSjlwwOd4L7u"  # Замените на ваш API ключ OpenAI
endpoint = "https://api.mistral.ai/v1/chat/completions"  # URL для API OpenAI
model_name = "mistral-small-2501"  # Имя модели
#model_name = "mistral-large-2411"  # Имя модели
output_dir = f"/app/datasets/oberon/qa/bb_ru/{model_name}"  # Имя выходного файла
os.makedirs(output_dir, exist_ok = True)



In [58]:
with open('/app/datasets/oberon/docs/bb_ru/Docu/CP-Lang.odc', 'r', encoding='utf-8') as text_file:
    text = text_file.read()
chunks = split_text_into_chunks(text,2,3)
for c in chunks[:2]:
    print('-' * 20)
    print(c)

--------------------
Copyright © 1994-2005 by Oberon microsystems, Inc., Switzerland.

All rights reserved. No part of this publication may be reproduced in any form or by any means, without prior written permission by Oberon microsystems except for the free electronic distribution of the unmodified document.

Oberon microsystems, Inc.
Technoparkstrasse 1
CH-8005 Zьrich
Switzerland

Oberon is a trademark of Prof. Niklaus Wirth.
Component Pascal is a trademark of Oberon microsystems, Inc.
All other trademarks and registered trademarks belong to their respective owners.

--------------------
Authors
	Oberon microsystems, Inc.
	March 2001

Authors of Oberon-2 report
	H. Mцssenbцck, N. Wirth
	Institut fьr Computersysteme, ETH Zьrich
	October 1993

Author of Oberon report
	N. Wirth
	Institut fьr Computersysteme, ETH Zьrich
	1987



In [65]:
res = generate_qa_pairs(chunks[0], api_key, endpoint, model_name)['choices'][0]['message']['content']

In [66]:
print(res)

[
 {"question": "Кто является владельцем копирайта документа?", "answer": "Oberon microsystems, Inc., Switzerland."},
 {"question": "В каком году начались права на копирайт?", "answer": "1994"},
 {"question": "Какие права имеют пользователи на документ?", "answer": "Пользователи могут свободно распространять неподредактированный документ в электронной форме."},
 {"question": "Какие действия требуют разрешения Oberon microsystems?", "answer": "Всякое использование, даже частичное, требует письменного разрешения от Oberon microsystems."},
 {"question": "Каков адрес Oberon microsystems?", "answer": "Technoparkstrasse 1, CH-8005 Zьrich, Switzerland"},
 {"question": "Кто является владельцем торговой марки Oberon?", "answer": "Prof. Niklaus Wirth"},
 {"question": "Кто является владельцем торговой марки Component Pascal?", "answer": "Oberon microsystems, Inc."}
]


In [48]:
process_text_files(directory, api_key, endpoint, model_name, output_dir, chunk_size, overlap)


Wrote /app/datasets/oberon/qa/bb_ru/mistral-small-2501.BB-Chars.odc.jsonl
Wrote /app/datasets/oberon/qa/bb_ru/mistral-small-2501.BB-Docu.odc.jsonl
Wrote /app/datasets/oberon/qa/bb_ru/mistral-small-2501.BB-License.odc.jsonl
Wrote /app/datasets/oberon/qa/bb_ru/mistral-small-2501.BB-Licensing-Policy.odc.jsonl
Wrote /app/datasets/oberon/qa/bb_ru/mistral-small-2501.BB-Open-Source-License.odc.jsonl
Wrote /app/datasets/oberon/qa/bb_ru/mistral-small-2501.BB-Road.odc.jsonl
Wrote /app/datasets/oberon/qa/bb_ru/mistral-small-2501.BB-Rules.odc.jsonl


KeyboardInterrupt: 